Code 

3D wheel

In [25]:
# Parameters
vf =  0.3
γ_evo = 0.1 
max_steps = 10
α_coeff = γ_evo*max_steps
iter_mod = 50
D = 3
mesh_name = "Wheel_3d.msh"
mesh_file = (@__DIR__)*"/Models/$mesh_name"

"c:\\Users\\IIT BBSR\\Desktop\\Amiya\\BTP\\Julia\\Level_Set_Method\\Cut_FEM\\3D_Wheel/Models/Wheel_3d.msh"

In [26]:
# Output Path
path = "../../../../../Result/3D_Wheel"

#  Directory to store visualization file
files_path = path*"_data/"
isdir(files_path) || mkpath(files_path)

#  Directory to store optimized model
model_path = path*"_model/"
isdir(model_path) || mkpath(model_path)

true

In [27]:
using GridapGmsh.GridapGmsh: GmshDiscreteModel
using Gridap.Geometry: UnstructuredDiscreteModel
using GridapTopOpt: update_labels!
using Gridap.Visualization: writevtk

In [28]:
# Load mesh
model = GmshDiscreteModel(mesh_file)
model = UnstructuredDiscreteModel(model)
f_diri(x) =
((cos(30π/180)<=x[1]<=cos(15π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(97.5π/180)<=x[1]<=cos(82.5π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(165π/180)<=x[1]<=cos(150π/180)) && abs(x[2] - sqrt(1-x[1]^2))<1e-4) ||
((cos(142.5π/180)<=x[1]<=cos(127.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4) ||
((cos(52.5π/180)<=x[1]<=cos(37.5π/180)) && abs(x[2] - -sqrt(1-x[1]^2))<1e-4)
update_labels!(1,model,f_diri,"Gamma_D_new")
writevtk(model,model_path*"model")

Info    : Reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'...
Info    : 15 entities
Info    : 219664 nodes
Info    : 1328560 elements                                                                                    
Info    : Done reading 'c:\Users\IIT BBSR\Desktop\Amiya\BTP\Julia\Level_Set_Method\Cut_FEM\3D_Wheel/Models/Wheel_3d.msh'


4-element Vector{Vector{String}}:
 ["../../../../../Result/3D_Wheel_model/model_0.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_1.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_2.vtu"]
 ["../../../../../Result/3D_Wheel_model/model_3.vtu"]

In [29]:
using Gridap.Geometry: Triangulation
using GridapTopOpt: get_element_diameter_field
using GridapTopOpt: get_element_diameters

In [30]:
# Get triangulation and element size
Ω_bg = Triangulation(model)
hₕ = get_element_diameter_field(model)
hmin = minimum(get_element_diameters(model));

In [31]:
using Gridap.FESpaces
using Gridap.ReferenceFEs

In [32]:
# Space
reffe_scalar = ReferenceFE(lagrangian,Float64,1)
V_φ = TestFESpace(model,reffe_scalar)       # V_φ = FESpace(model, ReferenceFE(lagrangian, Float64, 1))
V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N",])
U_reg = TrialFESpace(V_reg);

In [33]:
using Gridap.FESpaces: interpolate

In [34]:
# Cut the background model_f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
f1((x,y,z),q,r) = - cos(q*π*x)*cos(q*π*y)*cos(q*π*z)/q - r/q
f2((x,y,z)) = -sqrt(x^2+y^2)+0.9
φh = interpolate(x->min(f1(x,4,0.1),f2(x)),V_φ);

In [35]:
using GridapTopOpt

In [36]:
# Check LS
GridapTopOpt.correct_ls!(φh)

In [44]:
using Gridap
using GridapEmbedded.Interfaces: PHYSICAL
using GridapEmbedded.Interfaces: EmbeddedBoundary
using GridapEmbedded.Interfaces: GhostSkeleton
using GridapEmbedded.Interfaces: ACTIVE
using GridapDistributed: local_views

In [56]:
# Setup integration meshes and measures
order = 1
degree = 2*(order+1)
Γ_N = BoundaryTriangulation(model,tags=["Gamma_N",])
dΓ_N = Measure(Γ_N,degree)
dΩ_bg = Measure(Ω_bg,degree);
# Ω_data = EmbeddedCollection(model,φh) do cutgeo,cutgeo_facets,_φh
#     Ω = DifferentiableTriangulation(Triangulation(cutgeo,PHYSICAL),V_φ)
#     Γ  = DifferentiableTriangulation(EmbeddedBoundary(cutgeo),V_φ)
#     Γg = GhostSkeleton(cutgeo)
#     Ω_act = Triangulation(cutgeo,ACTIVE)
#     # Isolated volumes
#     φ_cell_values = map(get_cell_dof_values,local_views(_φh))
#     ψ,_ = get_isolated_volumes_mask_polytopal(model,φ_cell_values,["Gamma_D_new",])
#     (;
#         :Ω_act => Ω_act,
#         :Ω     => Ω,
#         :dΩ    => Measure(Ω,degree),
#         :Γg    => Γg,
#         :dΓg   => Measure(Γg,degree),
#         :n_Γg  => get_normal_vector(Γg),
#         :Γ     => Γ,
#         :dΓ    => Measure(Γ,degree),
#         :n_Γ   => get_normal_vector(Γ),
#         :ψ     => ψ
#     )
# end

GenericMeasure()

In [57]:
# Setup spaces
reffe_d = ReferenceFE(lagrangian,VectorValue{D,Float64},order)
function build_spaces(Ω_act)
    V = TestFESpace(Ω_act,reffe_d,conformity=:H1,dirichlet_tags=["Gamma_D_new",])
    U = TrialFESpace(V)
    return U,V
end;

### Weak form

In [58]:
# Material parameters
function lame_parameters(E,ν)
    λ = (E*ν)/((1+ν)*(1-2*ν))
    μ = E/(2*(1+ν))
    (λ, μ)
end
λs, μs = lame_parameters(1.0,0.3);

In [59]:
# Stabilization
α_Gd = 1e-7
k_d = 1.0
γ_Gd(h) = α_Gd*(λs + μs)*h^3

# Terms
σ(ε) = λs*tr(ε)*one(ε) + 2*μs*ε
a_s_Ω(d,s) = ε(s) ⊙ (σ ∘ ε(d)) # Elasticity
j_s_k(d,s) = mean(γ_Gd ∘ hₕ)*(jump(Ω_data.n_Γg ⋅ ∇(s)) ⋅ jump(Ω_data.n_Γg ⋅ ∇(d)))
v_s_ψ(d,s) = (k_d*Ω_data.ψ)*(d⋅s) # Isolated volume term
g((x,y,z)) = 100VectorValue(-y,x,0.0)

a(d,s,φ) = ∫(a_s_Ω(d,s) + v_s_ψ(d,s))Ω_data.dΩ + ∫(j_s_k(d,s))Ω_data.dΓg
l(s,φ) = ∫(s⋅g)dΓ_N;

## Optimisation functionals

In [60]:
vol_D = sum(∫(1)dΩ_bg)
J_comp(d,φ) = ∫(ε(d) ⊙ (σ ∘ ε(d)))Ω_data.dΩ
Vol(d,φ) = ∫(1/vol_D)Ω_data.dΩ - ∫(vf/vol_D)dΩ_bg
dVol(q,d,φ) = ∫(-1/vol_D*q/(abs(Ω_data.n_Γ ⋅ ∇(φ))))Ω_data.dΓ;

## Setup solver and FE operators

In [ ]:
# elast_ls = PETScLinearSolver()
# state_collection = EmbeddedCollection_in_φh(model,φh) do _φh
# update_collection!(Ω_data,_φh)
# U,V = build_spaces(Ω_data.Ω_act)
# state_map = AffineFEStateMap(a,l,U,V,V_φ;ls=elast_ls,adjoint_ls=elast_ls)
# (;
#     :state_map => state_map,
#     :J => StateParamMap(J_comp,state_map),
#     :C => map(Ci -> StateParamMap(Ci,state_map),[Vol,])
# )
# end